# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-4b3c2739ac-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Film Genre and composer explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the different film genre over years and the composers for the cinema.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q25089`| Woody Allen       | node |





Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for composer

3. Identify the BGP for film genre

4. Find how many films are been released in the United States of America from 2010-01-01 to 2015-31-12 for each film genre available (the result set must be genre IRI, label and #films).

5. Consider the timespan from 2001-01-01 to nowadays. Find the number of films released in this timespan divided by film genre and return only those with more than 50 films released every year (the result set must be genre IRI and label).

6. Consider the composers of Western films and their country of citizenship. Count the number of Western Films on which worked a composer grouped by country of citizenship and return the top-10 (the result set must be country IRI, label and #films).. 

7. Consider the decades from 1961 to 1970, and from 2001 to 2010 and compare the total number of western films released in each one (the result set must be two couples decade and #western films of that decade).

8. Consider the decades from 1961 to 1970, and from 2001 to 2010 and select only western films. Then, for each decade, compare the average cast members size per film (the result set must be two couples decade and average cast members size of that decade).

## Task 1
Identify the BGP for films

In [4]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q25089 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('name', 'Regensburg Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('name', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [5]:
# find occupation
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q25089 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('obj', 'http://www.wikidata.org/entity/Q15981151'), ('name', 'jazz musician')]
[('obj', 'http://www.wikidata.org/entity/Q18844224'), ('name', 'science fiction writer')]
[('obj', 'http://www.wikidata.org/entity/Q1930187'), ('name', 'journalist')]
[('obj', 'http://www.wikidata.org/entity/Q214917'), ('name', 'playwright')]
[('obj', 'http://www.wikidata.org/entity/Q245068'), ('name', 'comedian')]
[('obj', 'http://www.wikidata.org/entity/Q2526255'), ('name', 'film director')]
[('obj', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q3282637'), ('name', 'film producer')]
[('obj', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('obj', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('obj', 'http://www.wikidata.org/entity/Q482980'), ('name', 'author')]
[('obj', 'http://www.wikidata.org/entity/Q639669'), ('name', 'm

In [6]:
# explore film actor properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q10800557 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P508'), ('name', 'BNCF Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8814'), ('name',

In [7]:
# explore filmography
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q25089 wdt:P1283 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2455835'), ('name', 'Woody Allen filmography')]
1


In [8]:
# explore Woody Allen filomgraphy properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   wd:Q2455835 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]
6


In [9]:
# exploring is a list of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q2455835 wdt:P360 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


Final query for this task

In [10]:
# returning BGP for films
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q2455835 wdt:P360 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


## Task 2
Identify the BGP for composer

In [11]:
# find occupations of Woody Allen
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q25089 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('obj', 'http://www.wikidata.org/entity/Q15981151'), ('name', 'jazz musician')]
[('obj', 'http://www.wikidata.org/entity/Q18844224'), ('name', 'science fiction writer')]
[('obj', 'http://www.wikidata.org/entity/Q1930187'), ('name', 'journalist')]
[('obj', 'http://www.wikidata.org/entity/Q214917'), ('name', 'playwright')]
[('obj', 'http://www.wikidata.org/entity/Q245068'), ('name', 'comedian')]
[('obj', 'http://www.wikidata.org/entity/Q2526255'), ('name', 'film director')]
[('obj', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q3282637'), ('name', 'film producer')]
[('obj', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('obj', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('obj', 'http://www.wikidata.org/entity/Q482980'), ('name', 'author')]
[('obj', 'http://www.wikidata.org/entity/Q639669'), ('name', 'm

In [12]:
# find BGP of composer
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q25089 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name.
   FILTER( regex(?name, "^comp") )
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
1


Final query for this task

In [13]:
# find BGP of composer
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q25089 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name.
   FILTER( regex(?name, "^comp") )
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
1


## Task 3
Identify the BGP for film genre

In [14]:
# explore film properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   wd:Q11424 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1368'), ('name', 'LNB ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p',

In [15]:
# search instances of film
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   
   ?s wdt:P31 wd:Q11424 .
   # get the label
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q8178307'), ('name', '15 Park Avenue')]
[('s', 'http://www.wikidata.org/entity/Q8197668'), ('name', 'Idiot Love')]
[('s', 'http://www.wikidata.org/entity/Q319263'), ('name', 'Doctor in the House')]
[('s', 'http://www.wikidata.org/entity/Q8452990'), ('name', "Sunday League - Pepik Hnatek's Final Match")]
[('s', 'http://www.wikidata.org/entity/Q284184'), ('name', 'Ra.One')]
[('s', 'http://www.wikidata.org/entity/Q13637926'), ('name', 'Indra')]
[('s', 'http://www.wikidata.org/entity/Q13638153'), ('name', 'Inner Senses')]
[('s', 'http://www.wikidata.org/entity/Q13645807'), ('name', 'Madhumasam')]
[('s', 'http://www.wikidata.org/entity/Q13647258'), ('name', 'Marupakkam')]
[('s', 'http://www.wikidata.org/entity/Q13677566'), ('name', 'Timberjack')]
[('s', 'http://www.wikidata.org/entity/Q13693967'), ('name', 'Cross')]
[('s', 'http://www.wikidata.org/entity/Q15464247'), ('name', 'What the Fish')]
[('s', 'http://www.wikidata.org/entity/Q15475237'),

In [16]:
# explore properties of the film Thegidi
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   wd:Q15863744 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P2529'), ('name', 'ČSFD film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2603'), ('name', 'Kinopoisk film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3138'), ('name', 'OFDb film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3143'), ('name', 'elFilm film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P344'

In [17]:
# exploring genre
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q15863744 wdt:P136 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2484376'), ('name', 'thriller film')]
1


In [18]:
# exploring thriller film properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   
   wd:Q2484376 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3106'), ('name', 'Guardian topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3219'), ('name', 'Encyclopædia Universalis ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P4953'), ('name', 'Library of Congress Genre/Form Terms ID')]
[('p', 'http://www.wikidata.org/pr

In [19]:
# exploring instance of property
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q2484376 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q201658'), ('name', 'film genre')]
1


Final query for this task

In [20]:
# return BGP for film genre
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q2484376 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q201658'), ('name', 'film genre')]
1


## Task 4
Find how many films are been released in the United States of America from 2010-01-01 to 2015-31-12 for each film genre available (the result set must be genre IRI, label and #films).

In [21]:
# find the publication date of a film
queryString = """
SELECT DISTINCT ?obj
WHERE {
   
   wd:Q15863744 wdt:P577 ?obj .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '2014-01-01T00:00:00Z')]
1


In [22]:
# find the country of origin of a film
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   
   wd:Q15863744 wdt:P495 ?obj .
   #labels
   ?obj sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q668'), ('name', 'India')]
1


In [23]:
# find the movies released in the USA(wrong)
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   
   ?film wdt:P495 ?obj .
   #labels
   ?obj sc:name "United States of America"^^xsd:string .
   ?film sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [24]:
# find the movies released in the USA
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   
   ?film wdt:P495 ?obj .
   #labels
   ?obj sc:name ?o_name .
   ?film sc:name ?name .
   FILTER(regex(?o_name, "United States of America", "i"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q20966579'), ('name', 'Pokémon Go')]
[('film', 'http://www.wikidata.org/entity/Q228648'), ('name', '38 Special')]
[('film', 'http://www.wikidata.org/entity/Q300666'), ('name', 'Aaahh!!! Real Monsters')]
[('film', 'http://www.wikidata.org/entity/Q1269638'), ('name', 'Dälek')]
[('film', 'http://www.wikidata.org/entity/Q1575797'), ('name', 'Nicholas Jonas')]
[('film', 'http://www.wikidata.org/entity/Q2101436'), ('name', 'Ford C1 platform')]
[('film', 'http://www.wikidata.org/entity/Q2274504'), ('name', 'Love Boat Captain')]
[('film', 'http://www.wikidata.org/entity/Q3013333'), ('name', 'Dance Gavin Dance')]
[('film', 'http://www.wikidata.org/entity/Q33323'), ('name', 'Everybody Hates Chris')]
[('film', 'http://www.wikidata.org/entity/Q3644642'), ('name', 'Florida Georgia Line')]
[('film', 'http://www.wikidata.org/entity/Q500769'), ('name', 'Freaks and Geeks')]
[('film', 'http://www.wikidata.org/entity/Q609450'), ('name', 'Delinquent Habits

In [25]:
# find the movies released between 2010 and 2015
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   
   ?film wdt:P577 ?date .
   #labels
   ?film sc:name ?name .
   FILTER(?date >= "2010-01-01T00:00:00Z"^^xsd:dateTime && ?date < "2016-01-01T00:00:00Z"^^xsd:dateTime)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q10847276'), ('name', 'Green Garden Pop')]
[('film', 'http://www.wikidata.org/entity/Q306979'), ('name', 'Green a.Live')]
[('film', 'http://www.wikidata.org/entity/Q5409927'), ('name', 'Palabras del capitán cobarde')]
[('film', 'http://www.wikidata.org/entity/Q8139722'), ('name', 'Fight')]
[('film', 'http://www.wikidata.org/entity/Q8188317'), ('name', 'Adiós, adiós')]
[('film', 'http://www.wikidata.org/entity/Q8193988'), ('name', 'Alegría!')]
[('film', 'http://www.wikidata.org/entity/Q8350955'), ('name', 'Cordel Encantado')]
[('film', 'http://www.wikidata.org/entity/Q8452990'), ('name', "Sunday League - Pepik Hnatek's Final Match")]
[('film', 'http://www.wikidata.org/entity/Q97657741'), ('name', 'The Political Economy of the Animal Protection Policy in Taipei(1953-2013)')]
[('film', 'http://www.wikidata.org/entity/Q15102469'), ('name', 'A multi-domain approach to asteroid families identification')]
[('film', 'http://www.wikidata.org/ent

In [26]:
# find the movies released between 2010 and 2015 in the USA
queryString = """
SELECT DISTINCT ?film ?name ?o_name
WHERE {
   
   ?film wdt:P577 ?date ;
         wdt:P495 ?obj .
   #labels
   ?obj sc:name ?o_name .
   ?film sc:name ?name .
   FILTER((?date >= "2010-01-01T00:00:00Z"^^xsd:dateTime && ?date < "2016-01-01T00:00:00Z"^^xsd:dateTime) && regex(?o_name, "United States of America", "i"))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q15490985'), ('name', 'Jailhouse Heat'), ('o_name', 'United States of America')]
[('film', 'http://www.wikidata.org/entity/Q15046558'), ('name', 'Life After Beth'), ('o_name', 'United States of America')]
[('film', 'http://www.wikidata.org/entity/Q15596853'), ('name', 'Burying the Ex'), ('o_name', 'United States of America')]
[('film', 'http://www.wikidata.org/entity/Q15727557'), ('name', 'The Moon Song'), ('o_name', 'United States of America')]
[('film', 'http://www.wikidata.org/entity/Q15731057'), ('name', 'Feral'), ('o_name', 'United States of America')]
[('film', 'http://www.wikidata.org/entity/Q15731615'), ('name', 'Scribblenauts Unmasked: A DC Comics Adventure'), ('o_name', 'United States of America')]
[('film', 'http://www.wikidata.org/entity/Q15735273'), ('name', 'My Immortal'), ('o_name', 'United States of America')]
[('film', 'http://www.wikidata.org/entity/Q15735966'), ('name', 'Dani Daniels: Dare'), ('o_name', 'United States

In [27]:
# find the movies released between 2010 and 2015 in the USA group by genre and counting them
queryString = """
SELECT DISTINCT ?genre ?name (COUNT(?film) AS ?films)
WHERE {
   
   ?film wdt:P577 ?date ;
         wdt:P495 ?obj ;
         wdt:P136 ?genre .
   #labels
   ?obj sc:name ?o_name .
   ?genre sc:name ?name .
   FILTER((?date >= "2010-01-01T00:00:00Z"^^xsd:dateTime && ?date < "2016-01-01T00:00:00Z"^^xsd:dateTime) && regex(?o_name, "United States of America", "i"))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q1033891'), ('name', 'martial arts film'), ('films', '42')]
[('genre', 'http://www.wikidata.org/entity/Q130232'), ('name', 'drama'), ('films', '2608')]
[('genre', 'http://www.wikidata.org/entity/Q200092'), ('name', 'horror film'), ('films', '969')]
[('genre', 'http://www.wikidata.org/entity/Q858330'), ('name', 'romance novel'), ('films', '58')]
[('genre', 'http://www.wikidata.org/entity/Q10992055'), ('name', 'speculative fiction novel'), ('films', '176')]
[('genre', 'http://www.wikidata.org/entity/Q191489'), ('name', 'melodrama'), ('films', '43')]
[('genre', 'http://www.wikidata.org/entity/Q580850'), ('name', 'techno-thriller'), ('films', '28')]
[('genre', 'http://www.wikidata.org/entity/Q459435'), ('name', 'mockumentary'), ('films', '13')]
[('genre', 'http://www.wikidata.org/entity/Q186424'), ('name', 'detective fiction'), ('films', '13')]
[('genre', 'http://www.wikidata.org/entity/Q1535153'), ('name', 'superhero film'), ('films', '14

In [28]:
# find the movies released between 2010 and 2015 in the USA group by genre and counting them filtering out music genres and others
queryString = """
SELECT DISTINCT ?genre ?name (COUNT(?film) AS ?films)
WHERE {
   
   ?film wdt:P577 ?date ;
         wdt:P495 ?obj ;
         wdt:P136 ?genre .
         
   ?genre wdt:P31 wd:Q201658 .
   #labels
   ?obj sc:name ?o_name .
   ?genre sc:name ?name .
   FILTER((?date >= "2010-01-01T00:00:00Z"^^xsd:dateTime && ?date < "2016-01-01T00:00:00Z"^^xsd:dateTime) && regex(?o_name, "United States of America", "i"))
   
}
GROUP BY ?genre ?name
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q2593937'), ('name', 'conspiracy fiction'), ('films', '3')]
[('genre', 'http://www.wikidata.org/entity/Q3745430'), ('name', 'satirical film'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q1760864'), ('name', 'nature documentary'), ('films', '11')]
[('genre', 'http://www.wikidata.org/entity/Q1257444'), ('name', 'film adaptation'), ('films', '11')]
[('genre', 'http://www.wikidata.org/entity/Q104765957'), ('name', 'time-travel film'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q1919632'), ('name', 'melodrama'), ('films', '5')]
[('genre', 'http://www.wikidata.org/entity/Q130232'), ('name', 'drama'), ('films', '2608')]
[('genre', 'http://www.wikidata.org/entity/Q93204'), ('name', 'documentary film'), ('films', '1517')]
[('genre', 'http://www.wikidata.org/entity/Q7645884'), ('name', 'surf film'), ('films', '3')]
[('genre', 'http://www.wikidata.org/entity/Q3272147'), ('name', 'found footage'), ('films', '123')]


In [3]:
# find BGP for USA
queryString = """
SELECT DISTINCT ?obj ?o_name
WHERE {
   
   wd:Q20966579 wdt:P495 ?obj .
   #labels
   ?obj sc:name ?o_name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q30'), ('o_name', 'United States of America')]
1


Final query for this task

In [4]:
# find the movies released between 2010 and 2015 in the USA group by genre and counting them
queryString = """
SELECT DISTINCT ?genre ?name (COUNT(?film) AS ?films)
WHERE {
   
   ?film wdt:P577 ?date ;
         wdt:P495 wd:Q30 ;
         wdt:P136 ?genre .
         
   ?genre wdt:P31 wd:Q201658 .
   #labels
   ?genre sc:name ?name .
   FILTER((?date >= "2010-01-01T00:00:00Z"^^xsd:dateTime && ?date < "2016-01-01T00:00:00Z"^^xsd:dateTime))
   
}
GROUP BY ?genre ?name
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q130232'), ('name', 'drama'), ('films', '2608')]
[('genre', 'http://www.wikidata.org/entity/Q93204'), ('name', 'documentary film'), ('films', '1517')]
[('genre', 'http://www.wikidata.org/entity/Q188473'), ('name', 'action film'), ('films', '1329')]
[('genre', 'http://www.wikidata.org/entity/Q157443'), ('name', 'comedy film'), ('films', '1320')]
[('genre', 'http://www.wikidata.org/entity/Q200092'), ('name', 'horror film'), ('films', '969')]
[('genre', 'http://www.wikidata.org/entity/Q2484376'), ('name', 'thriller film'), ('films', '887')]
[('genre', 'http://www.wikidata.org/entity/Q471839'), ('name', 'science fiction film'), ('films', '779')]
[('genre', 'http://www.wikidata.org/entity/Q157394'), ('name', 'fantasy film'), ('films', '672')]
[('genre', 'http://www.wikidata.org/entity/Q959790'), ('name', 'crime film'), ('films', '530')]
[('genre', 'http://www.wikidata.org/entity/Q859369'), ('name', 'comedy-drama'), ('films', '495')]
[('genr

## Task 5
Consider the timespan from 2001-01-01 to nowadays. Find the number of films released in this timespan divided by film genre and return only those with more than 50 films released every year (the result set must be genre IRI and label).

In [30]:
# find all the films grouped by genre and year from 2001 to now (wrong)
queryString = """
SELECT DISTINCT ?genre YEAR(?date) (COUNT(?film) AS ?films)
WHERE {
   ?film wdt:P577 ?date ;
         wdt:P136 ?genre .
    FILTER(?date >="2001-01-01T00:00:00Z"^^xsd:dateTime && ?date <= NOW())
}
GROUP BY ?genre YEAR(?date)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q1051575'), ('callret-1', '2002'), ('films', '2')]
[('genre', 'http://www.wikidata.org/entity/Q34178'), ('callret-1', '2007'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q63860397'), ('callret-1', '2018'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q191067'), ('callret-1', '2015'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q272167'), ('callret-1', '2019'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q189125'), ('callret-1', '2017'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q12007812'), ('callret-1', '2003'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q332102'), ('callret-1', '2004'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q196999'), ('callret-1', '2014'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q189825'), ('callret-1', '2007'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q723418'), ('ca

In [31]:
# find all the films grouped by genre and year from 2001 to now filter out music genres
queryString = """
SELECT DISTINCT ?genre YEAR(?date) (COUNT(?film) AS ?films)
WHERE {
   ?film wdt:P577 ?date ;
         wdt:P136 ?genre .
   ?genre wdt:P31 wd:Q201658 .
   FILTER(?date >="2001-01-01T00:00:00Z"^^xsd:dateTime && ?date <= NOW())
}
GROUP BY ?genre YEAR(?date)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q11304653'), ('callret-1', '2020'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q496523'), ('callret-1', '2004'), ('films', '21')]
[('genre', 'http://www.wikidata.org/entity/Q63214877'), ('callret-1', '2002'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q63214877'), ('callret-1', '2003'), ('films', '2')]
[('genre', 'http://www.wikidata.org/entity/Q1033891'), ('callret-1', '2001'), ('films', '15')]
[('genre', 'http://www.wikidata.org/entity/Q1033891'), ('callret-1', '2017'), ('films', '6')]
[('genre', 'http://www.wikidata.org/entity/Q5258881'), ('callret-1', '2016'), ('films', '3')]
[('genre', 'http://www.wikidata.org/entity/Q25533274'), ('callret-1', '2005'), ('films', '3')]
[('genre', 'http://www.wikidata.org/entity/Q430525'), ('callret-1', '2007'), ('films', '2')]
[('genre', 'http://www.wikidata.org/entity/Q5258881'), ('callret-1', '2003'), ('films', '1')]
[('genre', 'http://www.wikidata.org/entity/Q4305

In [32]:
# find all the films grouped by genre and year from 2001 to now that are more than 50 for each year
queryString = """
SELECT DISTINCT ?genre YEAR(?date) (COUNT(?film) AS ?films)
WHERE {
   ?film wdt:P577 ?date ;
         wdt:P136 ?genre .
   ?genre wdt:P31 wd:Q201658 .
   FILTER(?date >="2001-01-01T00:00:00Z"^^xsd:dateTime && ?date <= NOW())
}
GROUP BY ?genre YEAR(?date)
HAVING (COUNT(?film) > 50)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q2143665'), ('callret-1', '2013'), ('films', '116')]
[('genre', 'http://www.wikidata.org/entity/Q2143665'), ('callret-1', '2010'), ('films', '119')]
[('genre', 'http://www.wikidata.org/entity/Q2143665'), ('callret-1', '2001'), ('films', '54')]
[('genre', 'http://www.wikidata.org/entity/Q2143665'), ('callret-1', '2018'), ('films', '65')]
[('genre', 'http://www.wikidata.org/entity/Q1054574'), ('callret-1', '2013'), ('films', '280')]
[('genre', 'http://www.wikidata.org/entity/Q1054574'), ('callret-1', '2018'), ('films', '231')]
[('genre', 'http://www.wikidata.org/entity/Q1054574'), ('callret-1', '2007'), ('films', '204')]
[('genre', 'http://www.wikidata.org/entity/Q130232'), ('callret-1', '2001'), ('films', '1010')]
[('genre', 'http://www.wikidata.org/entity/Q130232'), ('callret-1', '2003'), ('films', '1107')]
[('genre', 'http://www.wikidata.org/entity/Q130232'), ('callret-1', '2006'), ('films', '1377')]
[('genre', 'http://www.wikidata.or

In [33]:
# find all the genres from 2001 to now that have more than 50 films for each year
queryString = """
SELECT DISTINCT ?genre ?g_name 
WHERE{
    ?genre sc:name ?g_name ;
           wdt:P31 wd:Q201658 .
    
    {
        SELECT DISTINCT ?genre YEAR(?date) (COUNT(?film) AS ?films)
        WHERE {
           ?film wdt:P577 ?date ;
                 wdt:P136 ?genre .
           ?genre wdt:P31 wd:Q201658 .
           FILTER(?date >="2001-01-01T00:00:00Z"^^xsd:dateTime && ?date <= NOW())
        }
        GROUP BY ?genre YEAR(?date)
    }
    FILTER (?films > 50)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q1535153'), ('g_name', 'superhero film')]
[('genre', 'http://www.wikidata.org/entity/Q2143665'), ('g_name', "children's film")]
[('genre', 'http://www.wikidata.org/entity/Q28026639'), ('g_name', 'Christmas film')]
[('genre', 'http://www.wikidata.org/entity/Q1339864'), ('g_name', 'sport film')]
[('genre', 'http://www.wikidata.org/entity/Q1054574'), ('g_name', 'romance film')]
[('genre', 'http://www.wikidata.org/entity/Q130232'), ('g_name', 'drama')]
[('genre', 'http://www.wikidata.org/entity/Q157443'), ('g_name', 'comedy film')]
[('genre', 'http://www.wikidata.org/entity/Q185529'), ('g_name', 'pornographic film')]
[('genre', 'http://www.wikidata.org/entity/Q188473'), ('g_name', 'action film')]
[('genre', 'http://www.wikidata.org/entity/Q200092'), ('g_name', 'horror film')]
[('genre', 'http://www.wikidata.org/entity/Q369747'), ('g_name', 'war film')]
[('genre', 'http://www.wikidata.org/entity/Q52162262'), ('g_name', 'film based on litera

Final query for this task

In [34]:
# find all the genres from 2001 to now that have more than 50 films for each year
queryString = """
SELECT DISTINCT ?genre ?g_name 
WHERE{
    ?genre sc:name ?g_name ;
           wdt:P31 wd:Q201658 .
    
    {
        SELECT DISTINCT ?genre YEAR(?date) (COUNT(?film) AS ?films)
        WHERE {
           ?film wdt:P577 ?date ;
                 wdt:P136 ?genre .
           ?genre wdt:P31 wd:Q201658 .
           FILTER(?date >="2001-01-01T00:00:00Z"^^xsd:dateTime && ?date <= NOW())
        }
        GROUP BY ?genre YEAR(?date)
    }
    FILTER (?films > 50)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q1535153'), ('g_name', 'superhero film')]
[('genre', 'http://www.wikidata.org/entity/Q2143665'), ('g_name', "children's film")]
[('genre', 'http://www.wikidata.org/entity/Q28026639'), ('g_name', 'Christmas film')]
[('genre', 'http://www.wikidata.org/entity/Q1339864'), ('g_name', 'sport film')]
[('genre', 'http://www.wikidata.org/entity/Q1054574'), ('g_name', 'romance film')]
[('genre', 'http://www.wikidata.org/entity/Q130232'), ('g_name', 'drama')]
[('genre', 'http://www.wikidata.org/entity/Q157443'), ('g_name', 'comedy film')]
[('genre', 'http://www.wikidata.org/entity/Q185529'), ('g_name', 'pornographic film')]
[('genre', 'http://www.wikidata.org/entity/Q188473'), ('g_name', 'action film')]
[('genre', 'http://www.wikidata.org/entity/Q200092'), ('g_name', 'horror film')]
[('genre', 'http://www.wikidata.org/entity/Q369747'), ('g_name', 'war film')]
[('genre', 'http://www.wikidata.org/entity/Q52162262'), ('g_name', 'film based on litera

## Task 6
Consider the composers of Western films and their country of citizenship. Count the number of Western Films on which worked a composer grouped by country of citizenship and return the top-10 (the result set must be country IRI, label and #films)..

In [35]:
# find western films BGP
queryString = """
SELECT DISTINCT ?genre ?name
WHERE {
   ?film wdt:P136 ?genre .
   ?genre wdt:P31 wd:Q201658 .
   ?genre sc:name ?name . 
   FILTER(regex(?name, "^West", "i") )
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q172980'), ('name', 'Western film')]
1


In [36]:
# find composers of western films
queryString = """
SELECT DISTINCT ?composer ?name
WHERE {
   ?film wdt:P136 wd:Q172980 ;
         wdt:P86 ?composer ;
         wdt:P31 wd:Q11424 .
    ?composer sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('composer', 'http://www.wikidata.org/entity/Q2039003'), ('name', 'Boris Mokrousov')]
[('composer', 'http://www.wikidata.org/entity/Q1906085'), ('name', 'Marvin Hatley')]
[('composer', 'http://www.wikidata.org/entity/Q459556'), ('name', 'Carmen Dragon')]
[('composer', 'http://www.wikidata.org/entity/Q502923'), ('name', 'Dave Grusin')]
[('composer', 'http://www.wikidata.org/entity/Q1231447'), ('name', 'Walter Scharf')]
[('composer', 'http://www.wikidata.org/entity/Q212762'), ('name', 'Burt Bacharach')]
[('composer', 'http://www.wikidata.org/entity/Q3375624'), ('name', 'Perry Botkin Jr.')]
[('composer', 'http://www.wikidata.org/entity/Q365310'), ('name', 'Adolph Deutsch')]
[('composer', 'http://www.wikidata.org/entity/Q936212'), ('name', 'Fred Karlin')]
[('composer', 'http://www.wikidata.org/entity/Q1368185'), ('name', 'John P. Hammond')]
[('composer', 'http://www.wikidata.org/entity/Q716588'), ('name', 'Carlo Savina')]
[('composer', 'http://www.wikidata.org/entity/Q991563'), ('

In [37]:
# properties of one of the composers
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q1873497 ?p ?obj .
    ?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('name', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P1902'), ('name', 'Spotify artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1953'), ('name', 'Discogs artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2019'), ('name', 'AllMovie person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender')]
[('p', 'http://www.wikidata.org/prop/direct/P213'), ('name', 'ISNI')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2168'), ('name', 'Swedish Film Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/p

In [38]:
# find composers of western films with their country and take the top 10
queryString = """
SELECT DISTINCT ?country ?name (COUNT(?film) AS ?films)
WHERE {
   ?film wdt:P136 wd:Q172980 ;
         wdt:P86 ?composer ;
         wdt:P31 wd:Q11424 .
    ?composer wdt:P27 ?country .
    ?country sc:name ?name .
}
GROUP BY ?country ?name
ORDER BY DESC(?films)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America'), ('films', '1032')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('films', '54')]
[('country', 'http://www.wikidata.org/entity/Q34266'), ('name', 'Russian Empire'), ('films', '45')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('films', '40')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('films', '38')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('films', '36')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('name', 'Soviet Union'), ('films', '36')]
[('country', 'http://www.wikidata.org/entity/Q172579'), ('name', 'Kingdom of Italy'), ('films', '31')]
[('country', 'http://www.wikidata.org/entity/Q28513'), ('name', 'Austria-Hungary'), ('films', '31')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('films', '20')]
10


Final query for this task

In [39]:
# find composers of western films with their country and take the top 10
queryString = """
SELECT DISTINCT ?country ?name (COUNT(?film) AS ?films)
WHERE {
   ?film wdt:P136 wd:Q172980 ;
         wdt:P86 ?composer ;
         wdt:P31 wd:Q11424 .
    ?composer wdt:P27 ?country .
    ?country sc:name ?name .
}
GROUP BY ?country ?name
ORDER BY DESC(?films)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America'), ('films', '1032')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('films', '54')]
[('country', 'http://www.wikidata.org/entity/Q34266'), ('name', 'Russian Empire'), ('films', '45')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('films', '40')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('films', '38')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria'), ('films', '36')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('name', 'Soviet Union'), ('films', '36')]
[('country', 'http://www.wikidata.org/entity/Q172579'), ('name', 'Kingdom of Italy'), ('films', '31')]
[('country', 'http://www.wikidata.org/entity/Q28513'), ('name', 'Austria-Hungary'), ('films', '31')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('films', '20')]
10


## Task 7
Consider the decades from 1961 to 1970, and from 2001 to 2010 and compare the total number of western films released in each one (the result set must be two couples decade and #western films of that decade).

In [5]:
# find n_western films between 1961 and 1970
queryString = """
SELECT DISTINCT (GROUP_CONCAT(DISTINCT YEAR(?date); separator="-") AS ?decade) (COUNT(?film) AS ?films)
WHERE {
   ?film wdt:P136 wd:Q172980 ;
         wdt:P31 wd:Q11424 ;
         wdt:P577 ?date .
   FILTER(YEAR(?date) > 1960 && YEAR(?date) <= 1970)
}
GROUP BY (?decade)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('decade', '1961-1962-1963-1964-1965-1966-1967-1968-1969-1970'), ('films', '260')]
1


In [41]:
# find n_western films between 1961-1970 and 2001-2010 
queryString = """
SELECT DISTINCT ?decade ?films
WHERE {
    {
        SELECT DISTINCT ?decade (COUNT( DISTINCT ?film) AS ?films)
        WHERE {
           ?film wdt:P136 wd:Q172980 ;
                 wdt:P31 wd:Q11424 ;
                 wdt:P577 ?date .
           FILTER(YEAR(?date) > 1960 && YEAR(?date) <= 1970)
           BIND ("1961-1970" AS ?decade)
        }
    }
    UNION
    {
        SELECT DISTINCT ?decade (COUNT(DISTINCT ?film) AS ?films)
        WHERE {
           ?film wdt:P136 wd:Q172980 ;
                 wdt:P31 wd:Q11424 ;
                 wdt:P577 ?date .
           FILTER(YEAR(?date) > 2000 && YEAR(?date) <= 2010)
           BIND ("2001-2010" AS ?decade)
        }
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('decade', '1961-1970'), ('films', '250')]
[('decade', '2001-2010'), ('films', '64')]
2


In [8]:
# find n_western films between 1961-1970 and 2001-2010 without UNION (wrong)
queryString = """
SELECT DISTINCT ?decade ?films
WHERE {
    {
        SELECT DISTINCT ?decade (COUNT( DISTINCT ?film) AS ?films)
        WHERE {
           ?film wdt:P136 wd:Q172980 ;
                 wdt:P31 wd:Q11424 ;
                 wdt:P577 ?date .
           FILTER(YEAR(?date) > 1960 && YEAR(?date) <= 1970)
           BIND ("1961-1970" AS ?decade)
        }
        GROUP BY ?decade
    }
    {
        SELECT DISTINCT ?decade (COUNT(DISTINCT ?film) AS ?films)
        WHERE {
           ?film wdt:P136 wd:Q172980 ;
                 wdt:P31 wd:Q11424 ;
                 wdt:P577 ?date .
           FILTER(YEAR(?date) > 2000 && YEAR(?date) <= 2010)
           BIND ("2001-2010" AS ?decade)
        }
        GROUP BY ?decade
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [9]:
# find n_western films between 1961-1970 and 2001-2010 without UNION
queryString = """
SELECT DISTINCT ?decade (COUNT( DISTINCT ?film) AS ?films)
        WHERE {
           ?film wdt:P136 wd:Q172980 ;
                 wdt:P31 wd:Q11424 ;
                 wdt:P577 ?date .
           FILTER((YEAR(?date) > 1960 && YEAR(?date) <= 1970) || (YEAR(?date) > 2000 && YEAR(?date) <= 2010))
           BIND (if(YEAR(?date) > 1960 && YEAR(?date) <= 1970,"1961-1970", "2001-2010") AS ?decade)
        }
        GROUP BY ?decade
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('decade', '2001-2010'), ('films', '64')]
[('decade', '1961-1970'), ('films', '250')]
2


Final query for this task

In [10]:
# find n_western films between 1961-1970 and 2001-2010 
queryString = """
SELECT DISTINCT ?decade (COUNT( DISTINCT ?film) AS ?films)
        WHERE {
           ?film wdt:P136 wd:Q172980 ;
                 wdt:P31 wd:Q11424 ;
                 wdt:P577 ?date .
           FILTER((YEAR(?date) > 1960 && YEAR(?date) <= 1970) || (YEAR(?date) > 2000 && YEAR(?date) <= 2010))
           BIND (if(YEAR(?date) > 1960 && YEAR(?date) <= 1970,"1961-1970", "2001-2010") AS ?decade)
        }
        GROUP BY ?decade
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('decade', '2001-2010'), ('films', '64')]
[('decade', '1961-1970'), ('films', '250')]
2


## Task 8
Consider the decades from 1961 to 1970, and from 2001 to 2010 and select only western films. Then, for each decade, compare the average cast members size per film (the result set must be two couples decade and average cast members size of that decade).

In [43]:
# find cast members of a film
queryString = """
SELECT DISTINCT ?cast ?name
WHERE {
   
   wd:Q15863744 wdt:P161 ?cast .
   # get the label
   ?cast sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q16149455'), ('name', 'Ashok Selvan')]
1


In [11]:
# count cast members for western films between 1961 and 1970
queryString = """
SELECT DISTINCT ?decade ?film (COUNT(?cast) AS ?cast_size)
WHERE {
   ?film wdt:P136 wd:Q172980 ;
         wdt:P31 wd:Q11424 ;
         wdt:P577 ?date ;
         wdt:P161 ?cast .
   FILTER(YEAR(?date) > 1960 && YEAR(?date) <= 1970)
   BIND ("1961-1970" AS ?decade)
}
GROUP BY ?decade ?film
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('decade', '1961-1970'), ('film', 'http://www.wikidata.org/entity/Q7755669'), ('cast_size', '7')]
[('decade', '1961-1970'), ('film', 'http://www.wikidata.org/entity/Q7776761'), ('cast_size', '12')]
[('decade', '1961-1970'), ('film', 'http://www.wikidata.org/entity/Q614989'), ('cast_size', '6')]
[('decade', '1961-1970'), ('film', 'http://www.wikidata.org/entity/Q290069'), ('cast_size', '5')]
[('decade', '1961-1970'), ('film', 'http://www.wikidata.org/entity/Q2559902'), ('cast_size', '5')]
[('decade', '1961-1970'), ('film', 'http://www.wikidata.org/entity/Q16652691'), ('cast_size', '8')]
[('decade', '1961-1970'), ('film', 'http://www.wikidata.org/entity/Q1552534'), ('cast_size', '21')]
[('decade', '1961-1970'), ('film', 'http://www.wikidata.org/entity/Q1554665'), ('cast_size', '11')]
[('decade', '1961-1970'), ('film', 'http://www.wikidata.org/entity/Q3024718'), ('cast_size', '20')]
[('decade', '1961-1970'), ('film', 'http://www.wikidata.org/entity/Q1211060'), ('cast_size', '12')

In [12]:
# find average cast members size for western films between 1961-1970 and 2001-2010 
queryString = """
SELECT DISTINCT ?decade (AVG(?cast_size) AS ?avg_cast)
WHERE {
    {
        SELECT DISTINCT ?decade ?film (COUNT(?cast) AS ?cast_size)
        WHERE {
           ?film wdt:P136 wd:Q172980 ;
                 wdt:P31 wd:Q11424 ;
                 wdt:P577 ?date ;
                 wdt:P161 ?cast .
           FILTER(YEAR(?date) > 1960 && YEAR(?date) <= 1970)
           BIND ("1961-1970" AS ?decade)
        }
    }
    UNION
    {
        SELECT DISTINCT ?decade ?film (COUNT(?cast) AS ?cast_size)
        WHERE {
           ?film wdt:P136 wd:Q172980 ;
                 wdt:P31 wd:Q11424 ;
                 wdt:P577 ?date ;
                 wdt:P161 ?cast .
           FILTER(YEAR(?date) > 2000 && YEAR(?date) <= 2010)
           BIND ("2001-2010" AS ?decade)
        }
    }
}
GROUP BY ?decade
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('decade', '1961-1970'), ('avg_cast', '13.259414225941423')]
[('decade', '2001-2010'), ('avg_cast', '12.272727272727273')]
2


In [13]:
# find average cast members size for western films between 1961-1970 and 2001-2010 without UNION
queryString = """
SELECT DISTINCT ?decade (AVG(?cast_size) AS ?avg_cast)
WHERE {
    {
        SELECT DISTINCT ?decade ?film (COUNT(?cast) AS ?cast_size)
        WHERE {
           ?film wdt:P136 wd:Q172980 ;
                 wdt:P31 wd:Q11424 ;
                 wdt:P577 ?date ;
                 wdt:P161 ?cast .
           FILTER((YEAR(?date) > 1960 && YEAR(?date) <= 1970) || (YEAR(?date) > 2000 && YEAR(?date) <= 2010))
           BIND (if(YEAR(?date) > 1960 && YEAR(?date) <= 1970,"1961-1970","2001-2010") AS ?decade)
        }
    }
}
GROUP BY ?decade
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('decade', '2001-2010'), ('avg_cast', '12.272727272727273')]
[('decade', '1961-1970'), ('avg_cast', '13.259414225941423')]
2


Final query for this task

In [14]:
# find average cast members size for western films between 1961-1970 and 2001-2010 without UNION
queryString = """
SELECT DISTINCT ?decade (AVG(?cast_size) AS ?avg_cast)
WHERE {
    {
        SELECT DISTINCT ?decade ?film (COUNT(?cast) AS ?cast_size)
        WHERE {
           ?film wdt:P136 wd:Q172980 ;
                 wdt:P31 wd:Q11424 ;
                 wdt:P577 ?date ;
                 wdt:P161 ?cast .
           FILTER((YEAR(?date) > 1960 && YEAR(?date) <= 1970) || (YEAR(?date) > 2000 && YEAR(?date) <= 2010))
           BIND (if(YEAR(?date) > 1960 && YEAR(?date) <= 1970,"1961-1970","2001-2010") AS ?decade)
        }
    }
}
GROUP BY ?decade
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('decade', '2001-2010'), ('avg_cast', '12.272727272727273')]
[('decade', '1961-1970'), ('avg_cast', '13.259414225941423')]
2
